In [1]:
import numpy as np

class HillCipherECB:
    def __init__(self, key):
        self.key = key
        self.block_size = int(np.sqrt(len(key)))

    def encrypt(self, plaintext):
        # Pad plaintext with 'X' characters if necessary
        num_blocks = len(plaintext) // self.block_size + (len(plaintext) % self.block_size != 0)
        plaintext = plaintext.ljust(num_blocks * self.block_size, 'X')

        # Create key matrix
        key_matrix = self._create_key_matrix()

        # Encrypt plaintext blocks
        ciphertext = ''
        for i in range(num_blocks):
            block = plaintext[i*self.block_size:(i+1)*self.block_size]
            block_matrix = np.array([ord(c) - 65 for c in block], dtype=np.float64).reshape(self.block_size, 1)
            encrypted_block_matrix = np.mod(np.dot(key_matrix, block_matrix), 26)
            encrypted_block = ''.join([chr(int(c) + 65) for c in encrypted_block_matrix.flatten().tolist()])
            ciphertext += encrypted_block

        return ciphertext


    def decrypt(self, ciphertext):
        # Create key matrix
        key_matrix = self._create_key_matrix()

        # Decrypt ciphertext blocks
        plaintext = ''
        for i in range(len(ciphertext) // self.block_size):
            block = ciphertext[i*self.block_size:(i+1)*self.block_size]
            block_matrix = np.array([ord(c) - 65 for c in block], dtype=np.float64).reshape(self.block_size, 1)
            decrypted_block_matrix = np.mod(np.dot(np.linalg.inv(key_matrix), block_matrix), 26)
            decrypted_block = ''.join([chr(int(c) + 65) for c in decrypted_block_matrix.flatten().tolist()])
            plaintext += decrypted_block

        # Remove padding characters
        plaintext = plaintext.rstrip('X')

        return plaintext


    def _create_key_matrix(self):
        # Create key matrix from key string
        key_matrix = np.array([ord(c) - 65 for c in self.key]).reshape(self.block_size, self.block_size)
        return key_matrix

# Example usage
key = 'Usamaaaaa'
cipher = HillCipherECB(key)
plaintext = 'ibrahimwer'
ciphertext = cipher.encrypt(plaintext)
print('Ciphertext:', ciphertext)
decrypted_plaintext = cipher.decrypt(ciphertext)
print('Decrypted plaintext:', decrypted_plaintext)


Ciphertext: OQEWKQAGYGOY
Decrypted plaintext: BBXZABYXEZYD


In [21]:
import numpy as np
def pad(data):
    length = 3 - (len(data) % 3)
    data += chr(0) * length
    return data
def unpad(data):
    padding_length = ord(data[-1])
    if padding_length > len(data):
        raise ValueError("Invalid padding")
    padding = data[-padding_length:]
    if not all(c == padding_length for c in padding):
        raise ValueError("Invalid padding")
    return data[:-padding_length]


# Generate a key matrix for the Hill Cipher
def generate_key():
    while True:
        try:
            # Generate a random 3x3 matrix
            key = np.random.randint(0, 26, (3, 3))
            # Check if the matrix is invertible
            np.linalg.inv(key)
            return key
        except np.linalg.LinAlgError:
            continue
# Encrypt a plaintext block using the Hill Cipher
def encrypt_block(key, plaintext_block):
    plaintext_vector = np.array([ord(c.lower()) - 97 for c in plaintext_block])
    # Pad the plaintext vector with zeros to make it a multiple of 3
    pad_length = (3 - len(plaintext_vector) % 3) % 3
    plaintext_vector = np.pad(plaintext_vector, (0, pad_length), 'constant')
    plaintext_vector = plaintext_vector.reshape((-1, 3)).T
    # Multiply the key matrix by the plaintext vector
    ciphertext_vector = np.mod(key.dot(plaintext_vector), 26)
    # Convert the ciphertext vector to a string
    ciphertext_block = "".join([chr(c + 97) for c in ciphertext_vector.flat])
    return ciphertext_block


# CBC encryption function using Hill Cipher
def encrypt(key, iv, data):
    plaintext_blocks = [pad(data[i:i+3]) for i in range(0, len(data), 3)]
    ciphertext_blocks = [iv]

    for i, block in enumerate(plaintext_blocks):
        # XOR the current plaintext block with the previous ciphertext block 
        xor_block = [ord(c) ^ ord(d) for c, d in zip(block, ciphertext_blocks[i])]
        # Encrypt the XOR result using the Hill Cipher
        encrypted_block = encrypt_block(key, "".join([chr(c) for c in xor_block]))
        ciphertext_blocks.append(encrypted_block)

    # Concatenate all the encrypted blocks to get the final ciphertext
    ciphertext = "".join(ciphertext_blocks[1:])
    return ciphertext

def decrypt_block(key, ciphertext_block):
    ciphertext_vector = np.array([ord(c.lower()) - 97 for c in ciphertext_block])
    ciphertext_vector = ciphertext_vector.reshape((-1, 3)).T
    plaintext_vector = np.mod(np.linalg.inv(key).dot(ciphertext_vector), 26)
    plaintext_vector = plaintext_vector.astype(int)
    plaintext_block = "".join([chr(c + 97) for c in plaintext_vector.flat])
    plaintext_block = unpad(plaintext_block)
    return plaintext_block
# CBC decryption function using Hill Cipher
def decrypt(key, iv, ciphertext):
    ciphertext_blocks = [iv] + [ciphertext[i:i+3] for i in range(0, len(ciphertext), 3)]
    plaintext_blocks = []
    for i in range(1, len(ciphertext_blocks)):
        # Decrypt the ciphertext block using the Hill Cipher
        decrypted_block = decrypt_block(key, ciphertext_blocks[i])
        xor_block = [ord(c) ^ ord(d) for c, d in zip(decrypted_block, ciphertext_blocks[i-1])]
        
        plaintext_block = "".join([chr(c) for c in xor_block])
        plaintext_block = unpad(plaintext_block)  # unpad the plaintext block
        plaintext_blocks.append(plaintext_block)

    # Concatenate all the decrypted blocks to get the final plaintext
    plaintext = "".join(plaintext_blocks)

    return plaintext


# Generate a random 3x3 key matrix for the Hill Cipher
key = generate_key()

# Initialize an initialization vector (IV)
iv = "Car"

# Encrypt a message using CBC encryption with Hill Cipher
plaintext = "Hellooooo"
encrypted_data = encrypt(key, iv, plaintext)

# Print the results
print("Plaintext: ", plaintext)
print("Encrypted data: ", encrypted_data)
decrypted_data = decrypt(key, iv, encrypted_data)

print("Decrypted data: ", decrypted_data)
#print(decrypted_data.decode('utf-8'))

Plaintext:  Hellooooo
Encrypted data:  lqfrbqmxs


ValueError: Invalid padding

In [8]:
print(-ord('q'))

-113


In [90]:
import numpy as np

def hill_encrypt(key, plaintext):
    # Convert the plaintext to numbers
    plaintext = [ord(c) - ord('a') for c in plaintext.lower()]
    block_size = key.shape[0]
    # Pad the plaintext with 'x' if necessary
    if len(plaintext) % block_size != 0:
        plaintext += [23] * (block_size - len(plaintext) % block_size)
    # Split the plaintext into blocks
    plaintext_blocks = np.array(plaintext).reshape(-1, block_size)
    # Encrypt each block using Hill cipher
    ciphertext_blocks = np.matmul(plaintext_blocks, key) % 26
    # Convert the ciphertext back to letters
    ciphertext = ''.join([chr(c + ord('a')) for row in ciphertext_blocks for c in row])
    return ciphertext

def hill_decrypt(key, ciphertext):
    # Convert the ciphertext to numbers
    ciphertext = [ord(c) - ord('a') for c in ciphertext.lower()]
    block_size = key.shape[0]
    # Split the ciphertext into blocks
    ciphertext_blocks = np.array(ciphertext).reshape(-1, block_size)
    # Decrypt each block using Hill cipher
    inverse_key = np.linalg.inv(key)
    inverse_key = np.round(np.mod(inverse_key * np.linalg.det(key), 26)).astype(int)
    plaintext_blocks = np.matmul(ciphertext_blocks, inverse_key) % 26
    # Convert the plaintext back to letters
    plaintext = ''.join([chr(c + ord('a')) for row in plaintext_blocks for c in row])
    # Remove any padding
    plaintext = plaintext.rstrip('x')
    return plaintext

# Define the key
key = np.array([[3, 10, 20], [20, 9, 17], [9, 4, 17]])

# Define the plaintext and block size
plaintext = 'this is a test message'
block_size = key.shape[0]

# Encrypt the plaintext using ECB mode
ciphertext = ''
for i in range(0, len(plaintext), block_size):
    block = plaintext[i:i+block_size]
    block_ciphertext = hill_encrypt(key, block)
    ciphertext += block_ciphertext

# Decrypt the ciphertext using ECB mode
plaintext = ''
for i in range(0, len(ciphertext), block_size):
    block = ciphertext[i:i+block_size]
    block_plaintext = hill_decrypt(key, block)
    plaintext += block_plaintext

print('Plaintext:  ', plaintext)
print('Ciphertext: ', ciphertext)


Plaintext:   fvycnycnanfmcfnkmccasmrr
Ciphertext:  jzlwrpcljnfbfnusuqewudbe


In [30]:
import numpy as np

BLOCK_SIZE = 3

# Hill Cipher encryption function
def encrypt_block(key, plaintext_block):
    plaintext_vector = np.array([ord(c.lower()) - 97 for c in plaintext_block])
    plaintext_vector = plaintext_vector.reshape((-1, BLOCK_SIZE)).T
    ciphertext_vector = np.mod(key.dot(plaintext_vector), 26)
    ciphertext_vector = ciphertext_vector.astype(int)
    ciphertext_block = "".join([chr(c + 97) for c in ciphertext_vector.flat])
    return ciphertext_block

# CTR encryption function using Hill Cipher
def encrypt(key, iv, plaintext):
    iv = iv.encode('utf-8')
    plaintext_blocks = [plaintext[i:i+BLOCK_SIZE] for i in range(0, len(plaintext), BLOCK_SIZE)]
    ciphertext_blocks = []
    nonce = b'\x00\x00\x00'
    for i, plaintext_block in enumerate(plaintext_blocks):
        # Encrypt the nonce with the Hill Cipher
        keystream_block = encrypt_block(key, nonce.decode('utf-8'))

        # XOR the plaintext block with the keystream block to obtain the ciphertext block
        xor_block = [ord(c) ^ ord(d) for c, d in zip(plaintext_block, keystream_block)]

        # Append the ciphertext block to the list of ciphertext blocks
        ciphertext_blocks.append("".join([chr(c) for c in xor_block]))

        # Increment the nonce for the next block
        nonce = (int.from_bytes(nonce, byteorder='big') + 1).to_bytes(BLOCK_SIZE, byteorder='big')

    # Concatenate all the ciphertext blocks to get the final ciphertext
    ciphertext = "".join(ciphertext_blocks)

    # Prepend the IV to the ciphertext
    ciphertext = iv + ciphertext.encode('utf-8')

    return ciphertext
# CTR decryption function using Hill Cipher
def decrypt(key, iv, ciphertext):
    iv = iv.encode('utf-8')
    ciphertext = ciphertext[len(iv):]
    ciphertext_blocks = [ciphertext[i:i+BLOCK_SIZE] for i in range(0, len(ciphertext), BLOCK_SIZE)]
    plaintext_blocks = []
    nonce = b'\x00\x00\x00'
    for i, ciphertext_block in enumerate(ciphertext_blocks):
        # Encrypt the nonce with the Hill Cipher
        keystream_block = encrypt_block(key, nonce.decode('utf-8'))

        # XOR the ciphertext block with the keystream block to obtain the plaintext block
        xor_block = [ord(c) ^ ord(d) for c, d in zip(ciphertext_block, keystream_block)]

        # Append the plaintext block to the list of plaintext blocks
        plaintext_blocks.append("".join([chr(c) for c in xor_block]))

        # Increment the nonce for the next block
        nonce = (int.from_bytes(nonce, byteorder='big') + 1).to_bytes(BLOCK_SIZE, byteorder='big')

    # Concatenate all the plaintext blocks to get the final plaintext
    plaintext = "".join(plaintext_blocks)

    return plaintext

# Example key matrix for Hill Cipher
key = np.array([[3, 2, 1], [2, 1, 3], [1, 3, 2]])

# Example IV for CTR mode
iv = "abcdefgh"

# Example plaintext to be encrypted
plaintext = "attackatdawn"

# Encrypt the plaintext using Hill Cipher in CTR mode
ciphertext = encrypt(key, iv, plaintext)

# Print the ciphertext
print(ciphertext)
# Example ciphertext to be decrypted
ciphertext = b'abcdefghwbpwaf'

# Decrypt the ciphertext using Hill Cipher in CTR mode
plaintext = decrypt(key, iv, ciphertext)

# Print the plaintext
print(plaintext)



b'abcdefgh\x10\x05\x05\x13\x17\x18\x12\x03\x11\x15\r\x19'


TypeError: ord() expected string of length 1, but int found

In [2]:
import numpy as np

def pad(data):
    # Add padding to the data
    padding_length = (3 - len(data) % 3) % 3
    padding = chr(padding_length + 97) * padding_length
    return data + padding

def encrypt_block(key, plaintext_block):
    # Convert the plaintext block to a numpy array of numbers
    plaintext_vector = np.array([ord(c.lower()) - 97 for c in plaintext_block])
    # Reshape the vector into a 3x1 matrix
    plaintext_matrix = plaintext_vector.reshape((-1, 3)).T
    # Encrypt the block using the Hill Cipher
    ciphertext_matrix = np.mod(key.dot(plaintext_matrix), 26)
    # Convert the ciphertext matrix back to a vector of characters
    ciphertext_vector = ciphertext_matrix.T.flatten()
    ciphertext_block = "".join([chr(c + 97) for c in ciphertext_vector])
    # Return the encrypted block
    return ciphertext_block

def encrypt(key, plaintext):
    # Add padding to the plaintext
    padded_plaintext = pad(plaintext)
    # Split the padded plaintext into blocks of 3 characters
    plaintext_blocks = [padded_plaintext[i:i+3] for i in range(0, len(padded_plaintext), 3)]
    # Encrypt each block using the Hill Cipher
    ciphertext_blocks = [encrypt_block(key, block) for block in plaintext_blocks]
    # Concatenate the ciphertext blocks to get the final ciphertext
    ciphertext = "".join(ciphertext_blocks)
    # Return the ciphertext
    return ciphertext
key = np.array([[6, 24, 1], [13, 16, 10], [20, 17, 15]])
plaintext = "this is a secret message"
ciphertext = encrypt(key, plaintext)
print(ciphertext)


exvmczeajudciazwdbgagsgg


In [3]:
def decrypt(key, ciphertext):
    key_inverse = np.linalg.inv(key)
    key_determinant = int(np.round(np.linalg.det(key))) % 26
    plaintext = ""
    
    # Convert the ciphertext to a 3D numpy array
    ciphertext = np.array([[[ord(c) - 97]] for c in ciphertext])
    
    # Iterate over the 3D matrix in blocks of size (3,3,1)
    for i in range(0, ciphertext.shape[0], 3):
        for j in range(0, ciphertext.shape[1], 3):
            block = ciphertext[i:i+3, j:j+3, :]
            block_flat = block.reshape(-1, 1)
            
            # Apply the Hill Cipher decryption on the block
            block_decrypted = np.dot(key_inverse, block_flat)
            block_decrypted = np.mod(np.round(block_decrypted), 26).astype(int)
            block_decrypted = block_decrypted.reshape(block.shape)
            
            # Append the decrypted block to the plaintext
            plaintext += "".join([chr(b + 97) for b in block_decrypted[:,:,0].flatten()])
            
    return plaintext
key = np.array([[6, 24, 1], [13, 16, 10], [20, 17, 15]])
ciphertext="exvmczeajudciazwdbgagsgg"
# Decrypt the ciphertext using Hill Cipher in ECB mode
plaintext = decrypt(key, ciphertext)

# Print the plaintext
print(plaintext)


tbjnynfzvcaxoxmcbxezwbby


In [36]:
def pad(plaintext, block_size):
    padding_length = block_size - (len(plaintext) % block_size)
    padding = bytes([padding_length] * padding_length)
    return plaintext + padding

def unpad(plaintext):
    padding_length = plaintext[-1]
    return plaintext[:-padding_length]

def ecb_encrypt(plaintext, key):
    block_size = len(key)
    plaintext = pad(plaintext, block_size)
    ciphertext = b''
    for i in range(0, len(plaintext), block_size):
        block = plaintext[i:i+block_size]
        print(block)
        ciphertext += xor(block, key)
    return ciphertext

def ecb_decrypt(ciphertext, key):
    block_size = len(key)
    plaintext = b''
    
    for i in range(0, len(ciphertext), block_size):
        block = ciphertext[i:i+block_size]
        plaintext += xor(block, key)
    plaintext = unpad(plaintext)
    return plaintext

def xor(a, b):
    return bytes([x ^ y for x, y in zip(a, b)])


key = b'ibrahim'
plaintext = b'This is a secret message that needs to be encrypted.'
ciphertext = ecb_encrypt(plaintext, key)
print("Encrypted ciphertext:", ciphertext.hex())
decrypted_plaintext = ecb_decrypt(ciphertext, key)
print("Decrypted plaintext:", decrypted_plaintext.decode('utf-8'))



b'This is'
b' a secr'
b'et mess'
b'age tha'
b't needs'
b' to be '
b'encrypt'
b'ed.\x04\x04\x04\x04'
Encrypted ciphertext: 3d0a1b1248001e490352120d0a1f0c16520c0d1a1e080517411c010c1d421c040d0d1e49161d410a0c4d0c0c11131119190c065c656c6d69
Decrypted plaintext: This is a secret message that needs to be encrypted.


Message: mynameisibrahim
Input 4 letter cipher: hell
7
15
8
348
8
91
8
128
8
254
8
67
8
119
8
137
8
-103
KANAYOUWPRPQHMBR
Input 4 letter cipher: hell
7
15
MYNAMEISIBRAHIMJ


In [33]:
import math
import string
import sys

import numpy as np
from sympy import Matrix


# Shows the list of functions that can be run, and the user chooses one of them.
def option_menu():
    while True:
        print("---- HILL CIPHER (EXERCISE 3.2) " + "-"*200)
        print("List of available functions:")
        print("1) Encryption.")
        print("2) Decryption.")
        print("3) Known Plaintext Attack.")
        print("4) Exit.\n")
        try:
            chosen_function = int(input("\U0000270F Select the number of the function to run: "))
            if 1 <= chosen_function <= 4:
                return chosen_function
            else:
                print("\nYou must enter a number from 1 to 4\n")
        except ValueError:
            print("\nYou must enter a number from 1 to 4\n")
        input("Press Enter to continue.\n")


# Creates a dictionary, letters of the English alphabet to numbers, and returns it.
def get_from_letters_to_numbers():
    alphabet = {}
    for character in string.ascii_uppercase:
        alphabet[character] = string.ascii_uppercase.index(character)
    return alphabet


# Creates a dictionary, numbers to letters of the English alphabet, and returns it.
def get_from_numbers_to_letters():
    alphabet = get_from_letters_to_numbers()
    reverse_alphabet = {}
    for key, value in alphabet.items():
        reverse_alphabet[value] = key
    return reverse_alphabet


# Gets input from the user and checks if respects the alphabet.
def get_text_input(message, alphabet):
    while True:
        text = input(message)
        text = text.upper()

        # Checks that all characters in the text are letters.
        if all(keys in alphabet for keys in text):
            return text
        else:
            print("\nThe text must contain only characters from the english alphabet ([A to Z] or [a to z]), "
                  "without spaces and numbers.")


# Checks if the key is a square in length.
def is_square(key):
    key_length = len(key)
    if 2 <= key_length == int(math.sqrt(key_length)) ** 2:
        return True
    else:
        return False


# Creates the matrix k for the key.
def get_key_matrix(key, alphabet):
    k = list(key)
    m = int(math.sqrt(len(k)))
    for (i, character) in enumerate(k):
        k[i] = alphabet[character]

    # Reshape method transforms a one-dimensional array into a multi-dimensional matrix.
    return np.reshape(k, (m, m))


# Creates the matrix of m-grams of a text, if needed, complete the last m-gram with the last letter of the alphabet.
def get_text_matrix(text, m, alphabet):
    matrix = list(text)
    remainder = len(text) % m
    for (i, character) in enumerate(matrix):
        matrix[i] = alphabet[character]
    if remainder != 0:
        for i in range(m - remainder):
            # Adds 25 to the list because it corresponds to the letter Z.
            matrix.append(25)

    # Reshapes method transforms a one-dimensional array into a multi-dimensional matrix.
    return np.reshape(matrix, (int(len(matrix) / m), m)).transpose()


# Encrypts a Message and returns the ciphertext matrix.
def encrypt(key, plaintext, alphabet):
    # Takes the number of rows of the key.
    m = key.shape[0]

    # Takes the number of columns of the plaintext, which corresponds to the number of m-grams
    # into which the plaintext is divided.
    m_grams = plaintext.shape[1]

    # Encrypts the plaintext with the key provided k, calculate matrix c of ciphertext.
    ciphertext = np.zeros((m, m_grams)).astype(int)
    for i in range(m_grams):
        ciphertext[:, i] = np.reshape(np.dot(key, plaintext[:, i]) % len(alphabet), m)
    return ciphertext


# Transforms a matrix to a text, according to the alphabet.
def from_matrix_to_text(matrix, order, alphabet):
    # The plaintext and ciphertext matrices are read by columns.
    if order == 't':
        # The ravel() with order='F' transforms the matrix into an array by reading it by columns.
        text_array = np.ravel(matrix, order='F')

    # The key matrix is read by rows.
    else:
        # The ravel() method (by default order = C) transforms the matrix into an array by reading it by rows.
        text_array = np.ravel(matrix)
    text = ""
    for i in range(len(text_array)):
        text = text + alphabet[text_array[i]]
    return text


# Checks if the key is invertible and in that case returns the inverse of the matrix.
def get_inverse_matrix(matrix, alphabet):
    alphabet_len = len(alphabet)
    if math.gcd(int(round(np.linalg.det(matrix))), alphabet_len) == 1:
        matrix = Matrix(matrix)
        return np.matrix(matrix.inv_mod(alphabet_len))
    else:
        return None


# Decrypts a Message and returns the plaintext matrix.
def decrypt(key_inverse, ciphertext, alphabet):
    return encrypt(key_inverse, ciphertext, alphabet)


# Returns the value of m, inserted in input by the user during the known plaintext attack.
def get_m():
    while True:
        try:
            m = int(input("\U0000270F Insert the length of the grams (m): "))
            if m >= 2:
                return m
            else:
                print("\nYou must enter a number m >= 2\n")
        except ValueError:
            print("\nYou must enter a number m >= 2\n")


# Known Plaintext Attack
def known_plaintext_attack(ciphertext, plaintext_inverse, alphabet):
    return encrypt(ciphertext, plaintext_inverse, alphabet)


# Exposes all the functions for the various choices.
def main():
    while True:

        # Asks the user what function wants to run.
        choice = option_menu()

        # Gets two dictionaries, english alphabet to numbers and numbers to english alphabet.
        alphabet = get_from_letters_to_numbers()
        reverse_alphabet = get_from_numbers_to_letters()

        # Run the function selected by the user
        if choice == 1:

            # Asks the user the plaintext and the key for the encryption and checks the input.
            plaintext = get_text_input("\n\U0000270F Insert the plaintext: ", alphabet)
            key = get_text_input("\U0000270F Insert the key for encryption: ", alphabet)

            if is_square(key):

                # Gets the key matrix k.
                k = get_key_matrix(key, alphabet)
                print("\n\U00002022 Key Matrix:\n", k)

                # Gets the m-grams matrix p of the plaintext.
                p = get_text_matrix(plaintext, k.shape[0], alphabet)
                print("\U00002022 Plaintext Matrix:\n", p)

                # Encrypts the plaintext.
                c = encrypt(k, p, alphabet)

                # Transforms the ciphertext matrix to a text of the alphabet.
                ciphertext = from_matrix_to_text(c, "t", reverse_alphabet)

                print("\nThe message has been encrypted.\n")
                print("\U00002022 Generated Ciphertext Matrix:\n", c)
                print("\U00002022 Generated Ciphertext: ", ciphertext, "\n")
            else:
                print("\nThe length of the key must be a square and >= 2.\n")

        elif choice == 2:

            # Asks the user the ciphertext and the key for the encryption and checks the input.
            ciphertext = get_text_input("\n\U0000270F Insert the ciphertext: ", alphabet)
            key = get_text_input("\U0000270F Insert the key for decryption: ", alphabet)

            if is_square(key):

                # Gets the key matrix k.
                k = get_key_matrix(key, alphabet)

                # Checks if the key is invertible and in that case returns the inverse of the matrix.
                k_inverse = get_inverse_matrix(k, alphabet)

                if k_inverse is not None:

                    # Gets the m-grams matrix c of the ciphertext.
                    c = get_text_matrix(ciphertext, k_inverse.shape[0], alphabet)

                    print("\n\U00002022 Key Matrix:\n", k)
                    print("\U00002022 Ciphertext Matrix:\n", c)

                    # Decrypts the ciphertext.
                    p = decrypt(k_inverse, c, alphabet)

                    # Transforms the plaintext matrix to a text of the alphabet.
                    plaintext = from_matrix_to_text(p, "t", reverse_alphabet)

                    print("\nThe message has been decrypted.\n")
                    print("\U00002022 Generated Plaintext Matrix:\n", p)
                    print("\U00002022 Generated Plaintext: ", plaintext, "\n")
                else:
                    print("\nThe matrix of the key provided is not invertible.\n")
            else:
                print("\nThe key must be a square and size >= 2.\n")

        elif choice == 3:

            # Asks the user the text and the ciphertext to use them for the plaintext attack.
            plaintext = get_text_input("\n\U0000270F Insert the plaintext for the attack: ", alphabet)
            ciphertext = get_text_input("\U0000270F Insert the ciphertext of the plaintext for the attack: ", alphabet)

            # Asks the user the length of the grams
            m = get_m()

            if len(plaintext) / m >= m:

                # Gets the m-grams matrix p of the plaintext and takes the firsts m.
                p = get_text_matrix(plaintext, m, alphabet)
                # Takes all rows and only the first m columns.
                p = p[:, 0:m]

                # Checks if the matrix of the plaintext is invertible and in that case returns the inverse of the
                # matrix.
                p_inverse = get_inverse_matrix(p, alphabet)

                if p_inverse is not None:

                    # Gets the m-grams matrix c of the ciphertext.
                    c = get_text_matrix(ciphertext, m, alphabet)
                    # Takes all rows and only the first m columns.
                    c = c[:, 0:m]

                    if c.shape[1] == p.shape[0]:
                        print("\n\U00002022 Ciphertext Matrix C*:\n", c)
                        print("\U00002022 Plaintext Matrix P*:\n", p)

                        # Forces the ciphertext provided.
                        k = known_plaintext_attack(c, p_inverse, alphabet)

                        # Transforms the key matrix to a text of the alphabet.
                        key = from_matrix_to_text(k, "k", reverse_alphabet)

                        print("\nThe key has been found.\n")
                        print("\U00002022 Generated Key Matrix:\n", k)
                        print("\U00002022 Generated Key: ", key, "\n")

                    else:
                        print("\nThe number of m-grams for plaintext and ciphertext are different.\n")
                else:
                    print("\nThe matrix of the plaintext provided is not invertible.\n")
            else:
                print("\nThe length of the plaintext must be compatible with the length of the grams (m).\n")

        elif choice == 4:
            sys.exit(0)
        input("Press Enter to return to the selection menu.\n")


if __name__ == '__main__':
    main()

---- HILL CIPHER (EXERCISE 3.2) --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
List of available functions:
1) Encryption.
2) Decryption.
3) Known Plaintext Attack.
4) Exit.

✏ Select the number of the function to run: 1

✏ Insert the plaintext: help
✏ Insert the key for encryption: hell

• Key Matrix:
 [[ 7  4]
 [11 11]]
• Plaintext Matrix:
 [[ 7 11]
 [ 4 15]]

The message has been encrypted.

• Generated Ciphertext Matrix:
 [[13  7]
 [17  0]]
• Generated Ciphertext:  NRHA 

Press Enter to return to the selection menu.
2
---- HILL CIPHER (EXERCISE 3.2) --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
List of available functions:
1) Encryption.
2) Decryption.
3) Known Plaintext Attack.

KeyboardInterrupt: Interrupted by user

In [52]:
def pad(plaintext, block_size):
    padding_length = block_size - (len(plaintext) % block_size)
    padding = bytes([padding_length] * padding_length)
    return plaintext + padding

def unpad(plaintext):
    padding_length = plaintext[-1]
    return plaintext[:-padding_length]

def ecb_encrypt(plaintext, key):
    block_size = len(key)
    C=make_key(key)
    plaintext = pad(plaintext, block_size)
    ciphertext = ''
    for i in range(0, len(plaintext), block_size):
        block = plaintext[i:i+block_size]
        ciphertext = encrypt(msg,C)
#         ciphertext += xor(block, key)
    return ciphertext

def ecb_decrypt(ciphertext, key):
    key = key.encode('utf-8')
    C=make_key(key)
    block_size = len(key)
    plaintext = b''
    for i in range(0, len(ciphertext), block_size):
        block = ciphertext[i:i+block_size]
#         plaintext += xor(block, key)
        
    plaintext = unpad(plaintext)
    return plaintext

def xor(a, b):
    return bytes([x ^ y for x, y in zip(a, b)])
import numpy as np


def encrypt(msg,C):
    print(C)
    print(msg)
    # Replace spaces with nothing
    msg = msg.replace(" ", "")
    # Ask for keyword and get encryption matrix
    
    # Append zero if the messsage isn't divisble by 2
    len_check = len(msg) % 2 == 0
    if not len_check:
        msg += "0"
    # Populate message matrix
    P = create_matrix_of_integers_from_string(msg)
    # Calculate length of the message
    msg_len = int(len(msg) / 2)
    # Calculate P * C
    encrypted_msg = ""

    for i in range(msg_len):
        # Dot product
        
        row_0 = P[0][i] * C[0][0] + P[1][i] * C[0][1]
        # Modulate and add 65 to get back to the A-Z range in ascii
        integer = int(row_0 % 26 + 65)
        # Change back to chr type and add to text
        encrypted_msg += chr(integer)
        # Repeat for the second column
        row_1 = P[0][i] * C[1][0] + P[1][i] * C[1][1]
        integer = int(row_1 % 26 + 65)
        encrypted_msg += chr(integer)
    return encrypted_msg

def decrypt(encrypted_msg):
    # Ask for keyword and get encryption matrix
    C = make_key()
    # Inverse matrix
    determinant = C[0][0] * C[1][1] - C[0][1] * C[1][0]
    determinant = determinant % 26
    multiplicative_inverse = find_multiplicative_inverse(determinant)
    C_inverse = C
    # Swap a <-> d
    C_inverse[0][0], C_inverse[1][1] = C_inverse[1, 1], C_inverse[0, 0]
    # Replace
    C[0][1] *= -1
    C[1][0] *= -1
    for row in range(2):
        for column in range(2):
            C_inverse[row][column] *= multiplicative_inverse
            C_inverse[row][column] = C_inverse[row][column] % 26

    P = create_matrix_of_integers_from_string(encrypted_msg)
    msg_len = int(len(encrypted_msg) / 2)
    decrypted_msg = ""
    for i in range(msg_len):
        # Dot product
        column_0 = P[0][i] * C_inverse[0][0] + P[1][i] * C_inverse[0][1]
        # Modulate and add 65 to get back to the A-Z range in ascii
        integer = int(column_0 % 26 + 65)
        # Change back to chr type and add to text
        decrypted_msg += chr(integer)
        # Repeat for the second column
        column_1 = P[0][i] * C_inverse[1][0] + P[1][i] * C_inverse[1][1]
        integer = int(column_1 % 26 + 65)
        decrypted_msg += chr(integer)
    if decrypted_msg[-1] == "0":
        decrypted_msg = decrypted_msg[:-1]
    return decrypted_msg

def find_multiplicative_inverse(determinant):
    multiplicative_inverse = -1
    for i in range(26):
        inverse = determinant * i
        if inverse % 26 == 1:
            multiplicative_inverse = i
            break
    return multiplicative_inverse


def make_key(cipher):
     # Make sure cipher determinant is relatively prime to 26 and only a/A - z/Z are given
    determinant = 0
    C = None
    while True:
#         cipher = input("Input 4 letter cipher: ")
        C = create_matrix_of_integers_from_string(cipher)
        determinant = C[0][0] * C[1][1] - C[0][1] * C[1][0]
        determinant = determinant % 26
        print(determinant)
        inverse_element = find_multiplicative_inverse(determinant)
        print(inverse_element)
        if inverse_element == -1:
            print("Determinant is not relatively prime to 26, uninvertible key")
        elif np.amax(C) > 26 and np.amin(C) < 0:
            print("Only a-z characters are accepted")
            print(np.amax(C), np.amin(C))
        else:
            break
    return C

def create_matrix_of_integers_from_string(string):
    # Map string to a list of integers a/A <-> 0, b/B <-> 1 ... z/Z <-> 25
    integers = [chr_to_int(c) for c in string]
    length = len(integers)
    M = np.zeros((2, int(length / 2)), dtype=np.int32)
    iterator = 0
    for column in range(int(length / 2)):
        for row in range(2):
            M[row][column] = integers[iterator]
            iterator += 1
    return M

def chr_to_int(char):
    # Uppercase the char to get into range 65-90 in ascii table
    char = char.upper()
    # Cast chr to int and subtract 65 to get 0-25
    integer = ord(char) - 65
    return integer

# if __name__ == "__main__":
#     msg = input("Message: ")
#     encrypted_msg = encrypt(msg)
#     print(encrypted_msg)
#     decrypted_msg = decrypt(encrypted_msg)
#     print(decrypted_msg)

key = 'ibrahim'
plaintext = 'This is a secret message that needs to be encrypted.'
ciphertext = ecb_encrypt(plaintext.encode('utf-8'), key)
print("Encrypted ciphertext:", ciphertext.hex())
decrypted_plaintext = ecb_decrypt(ciphertext, key)
print("Decrypted plaintext:", decrypted_plaintext.decode('utf-8'))


9
3
[[ 8 17  7]
 [ 1  0  8]]
mynameisibrahim
[[ 8 17  7]
 [ 1  0  8]]
mynameisibrahim
[[ 8 17  7]
 [ 1  0  8]]
mynameisibrahim
[[ 8 17  7]
 [ 1  0  8]]
mynameisibrahim
[[ 8 17  7]
 [ 1  0  8]]
mynameisibrahim
[[ 8 17  7]
 [ 1  0  8]]
mynameisibrahim
[[ 8 17  7]
 [ 1  0  8]]
mynameisibrahim
[[ 8 17  7]
 [ 1  0  8]]
mynameisibrahim


AttributeError: 'str' object has no attribute 'hex'

In [58]:
# CBC encryption
def encrypt_CBC(plaintext, key, iv):
    block_size = len(key)
    plaintext = pad(plaintext, block_size)
    ciphertext = b""
    prev_block = iv
    for i in range(0, len(plaintext), block_size):
        block = xor(plaintext[i:i+block_size], prev_block)
        prev_block = encrypt_block(block, key)
        ciphertext += prev_block
    return ciphertext

# CBC decryption
def decrypt_CBC(ciphertext, key, iv):
    block_size = len(key)
    plaintext = b""
    prev_block = iv
    for i in range(0, len(ciphertext), block_size):
        block = decrypt_block(ciphertext[i:i+block_size], key)
        plaintext += xor(block, prev_block)
        prev_block = ciphertext[i:i+block_size]
    return unpad(plaintext)

# Padding functions
def pad(data, block_size):
    padding_size = block_size - len(data) % block_size
    padding = bytes([padding_size]) * padding_size
    return data + padding

def unpad(data):
    padding_size = data[-1]
    if padding_size > len(data):
        raise ValueError("Invalid padding")
    return data[:-padding_size]

# XOR function
def xor(a, b):
    return bytes([x ^ y for x, y in zip(a, b)])
import numpy as np


def encrypt_block(msg,C):
    # Replace spaces with nothing
    msg = msg.replace(" ", "")
    # Ask for keyword and get encryption matrix
    
    # Append zero if the messsage isn't divisble by 2
    len_check = len(msg) % 2 == 0
    if not len_check:
        msg += "0"
    # Populate message matrix
    P = create_matrix_of_integers_from_string(msg)
    # Calculate length of the message
    msg_len = int(len(msg) / 2)
    # Calculate P * C
    encrypted_msg = ""

    for i in range(msg_len):
        # Dot product
        
        row_0 = P[0][i] * C[0][0] + P[1][i] * C[0][1]
        # Modulate and add 65 to get back to the A-Z range in ascii
        integer = int(row_0 % 26 + 65)
        # Change back to chr type and add to text
        encrypted_msg += chr(integer)
        # Repeat for the second column
        row_1 = P[0][i] * C[1][0] + P[1][i] * C[1][1]
        integer = int(row_1 % 26 + 65)
        encrypted_msg += chr(integer)
    return encrypted_msg

def decrypt_block(encrypted_msg,C):
    # Ask for keyword and get encryption matrix
    # Inverse matrix
    determinant = C[0][0] * C[1][1] - C[0][1] * C[1][0]
    determinant = determinant % 26
    multiplicative_inverse = find_multiplicative_inverse(determinant)
    C_inverse = C
    # Swap a <-> d
    C_inverse[0][0], C_inverse[1][1] = C_inverse[1, 1], C_inverse[0, 0]
    # Replace
    C[0][1] *= -1
    C[1][0] *= -1
    for row in range(2):
        for column in range(2):
            C_inverse[row][column] *= multiplicative_inverse
            C_inverse[row][column] = C_inverse[row][column] % 26

    P = create_matrix_of_integers_from_string(encrypted_msg)
    msg_len = int(len(encrypted_msg) / 2)
    decrypted_msg = ""
    for i in range(msg_len):
        # Dot product
        column_0 = P[0][i] * C_inverse[0][0] + P[1][i] * C_inverse[0][1]
        # Modulate and add 65 to get back to the A-Z range in ascii
        integer = int(column_0 % 26 + 65)
        # Change back to chr type and add to text
        decrypted_msg += chr(integer)
        # Repeat for the second column
        column_1 = P[0][i] * C_inverse[1][0] + P[1][i] * C_inverse[1][1]
        integer = int(column_1 % 26 + 65)
        decrypted_msg += chr(integer)
    if decrypted_msg[-1] == "0":
        decrypted_msg = decrypted_msg[:-1]
    return decrypted_msg

def find_multiplicative_inverse(determinant):
    multiplicative_inverse = -1
    for i in range(26):
        inverse = determinant * i
        if inverse % 26 == 1:
            multiplicative_inverse = i
            break
    return multiplicative_inverse


def make_key(cipher):
     # Make sure cipher determinant is relatively prime to 26 and only a/A - z/Z are given
    determinant = 0
    C = None
    while True:
#         cipher = input("Input 4 letter cipher: ")
        C = create_matrix_of_integers_from_string(cipher)
        determinant = C[0][0] * C[1][1] - C[0][1] * C[1][0]
        determinant = determinant % 26
        
        inverse_element = find_multiplicative_inverse(determinant)
        
        if inverse_element == -1:
            print("Determinant is not relatively prime to 26, uninvertible key")
        elif np.amax(C) > 26 and np.amin(C) < 0:
            print("Only a-z characters are accepted")
            print(np.amax(C), np.amin(C))
        else:
            break
    return C

def create_matrix_of_integers_from_string(string):
    # Map string to a list of integers a/A <-> 0, b/B <-> 1 ... z/Z <-> 25
    integers = [chr_to_int(c) for c in string]
    length = len(integers)
    M = np.zeros((2, int(length / 2)), dtype=np.int32)
    iterator = 0
    for column in range(int(length / 2)):
        for row in range(2):
            M[row][column] = integers[iterator]
            iterator += 1
    return M

def chr_to_int(char):
    # Uppercase the char to get into range 65-90 in ascii table
    char = char.upper()
    # Cast chr to int and subtract 65 to get 0-25
    integer = ord(char) - 65
    return integer

# Example usage
plaintext = b"Hello world!"
key = b"supersecretkey12"
iv = b"initialvector123"
ciphertext = encrypt_CBC(plaintext, key, iv)
print("Ciphertext:", ciphertext)
decrypted = decrypt_CBC(ciphertext, key, iv)
print("Decrypted:", decrypted)


b'!\x0b\x05\x18\x06A\x1b\x19\x17\x0f\x10Nv567'
b'supersecretkey12'


ValueError: Invalid block size

In [64]:
def create_matrix_of_integers_from_byte_string(byte_string):
    # Decode byte string into string using utf-8 encoding
    string = byte_string.decode('utf-8')
    # Remove non-numeric characters and convert to list of integers
    integers = [int(c) for c in string if c.isnumeric()]
    # Reshape list of integers into 2x2 matrix
    matrix = [integers[:2], integers[2:]]
    return matrix

def create_matrix_of_integers_from_string(string):
    # Remove non-numeric characters and convert to list of integers
    integers = [ord(c) - 65 for c in string if c.isalpha()]
    # Reshape list of integers into 2x2 matrix
    matrix = [integers[:2], integers[2:]]
    return matrix
def hill_cipher_encrypt_block(msg_bytes, key):
    # Convert the key to a 2x2 matrix of integers
    key_matrix = create_matrix_of_integers_from_string(key)
    
    # If the length of the message is not even, append a null byte
    if len(msg_bytes) % 2 != 0:
        msg_bytes += b'\x00'
    
    # Convert the message to a string of hexadecimal digits
    msg_hex = msg_bytes.hex()
    
    # Convert the hexadecimal string to a list of integers
    msg_ints = [int(msg_hex[i:i+2], 16) for i in range(0, len(msg_hex), 2)]
    
    # Convert the list of integers to a matrix of size (2 x len(msg_ints)/2)
    msg_matrix = [msg_ints[i:i+2] for i in range(0, len(msg_ints), 2)]
    
    # Encrypt the message by multiplying the key matrix and the message matrix
    # and reducing the result modulo 26
    enc_matrix = [[sum(x * y for x, y in zip(row, col)) % 26 for col in zip(*key_matrix)] for row in msg_matrix]
    
    # Convert the encrypted matrix to a string of uppercase letters
    enc_str = "".join(chr(c + 65) for row in enc_matrix for c in row)
    
    return enc_str


msg = b'\x01\x07\x1e\x11'

key = b"myke"
encrypted_msg = hill_cipher_encrypt_block(msg, key)
print(encrypted_msg)


AWGE


In [72]:


def hill_cipher_encrypt_block(msg_bytes, key):
    # Convert the key to a 2x2 matrix of integers
    key_matrix = create_matrix_of_integers_from_string(key)
    
    # If the length of the message is not even, append a null byte
    if len(msg_bytes) % 2 != 0:
        msg_bytes += b'\x00'
    
    # Convert the message to a string of hexadecimal digits
    msg_hex = msg_bytes.hex()
    
    # Convert the hexadecimal string to a list of integers
    msg_ints = [int(msg_hex[i:i+2], 16) for i in range(0, len(msg_hex), 2)]
    
    # Convert the list of integers to a matrix of size (2 x len(msg_ints)/2)
    msg_matrix = [[msg_ints[i], msg_ints[i+1]] for i in range(0, len(msg_ints), 2)]
    
    # Encrypt the message by multiplying the key matrix and the message matrix
    # and reducing the result modulo 26
    enc_matrix = [[sum(x * y for x, y in zip(row, col)) % 26 for col in zip(*key_matrix)] for row in msg_matrix]
    
    # Convert the encrypted matrix to a string of uppercase letters
    enc_str = "".join(chr(c + 65) for row in enc_matrix for c in row)
    
    return enc_str.encode('utf-8')
msg = b'ibraibra'
key = "hell"


b'PMLHPMLH'


In [88]:
def create_matrix_of_integers_from_byte_string(byte_string):
    # Decode byte string into string using utf-8 encoding
    string = byte_string.decode('utf-8')
    # Remove non-numeric characters and convert to list of integers
    integers = [int(c) for c in string if c.isnumeric()]
    # Reshape list of integers into 2x2 matrix
    matrix = [integers[:2], integers[2:]]
    return matrix


def create_matrix_of_integers_from_string(string):
    # Remove non-numeric characters and convert to list of integers
    integers = [ord(c) - 65 for c in string if c.isalpha()]
    # Reshape list of integers into 2x2 matrix
    matrix = [[integers[i], integers[i+1]] for i in range(0, len(integers), 2)]
    return matrix


def hill_cipher_decrypt_block(enc_msg_bytes, key):
    key_matrix = create_matrix_of_integers_from_string(key)
    enc_msg = enc_msg_bytes.decode('utf-8')
    enc_ints = [ord(c) - 65 for c in enc_msg if c.isalpha()]
    enc_matrix = [enc_ints[i:i+2] for i in range(0, len(enc_ints), 2)]
    key_det = key_matrix[0][0] * key_matrix[1][1] - key_matrix[0][1] * key_matrix[1][0]
    key_inv = [[key_matrix[1][1], -key_matrix[0][1]], [-key_matrix[1][0], key_matrix[0][0]]]
    key_inv = [[(x * key_det) % 26 for x in row] for row in key_inv]
    dec_matrix = [[sum(x * y for x, y in zip(row, col)) % 26 for col in zip(*key_inv)] for row in enc_matrix]
    dec_str = "".join(chr(c ) for row in dec_matrix for c in row)
    
    return dec_str
msg = b'ibraibra'
key = "heww"
encrypted_msg = hill_cipher_encrypt_block(msg, key)
print(encrypted_msg)
encrypted_msg = hill_cipher_decrypt_block(encrypted_msg, key)
print(encrypted_msg)

b'BYMIBYMI'



